In [4]:
import re
import pandas as pd
import numpy as np
from Bio.SeqUtils import ProtParam

#  Open file

In [5]:
def load_file(filename):
    seq_tuple = []
    count = 0
    with open(filename) as f:
        header = ""
        sequence = ""
        for line in f:
            line = line.strip()  #remove white spaces
            
            if line.startswith('>'):
                if sequence != '':
                    #to filter sequences by their length
#                     seq_len = len(sequence)
#                     if seq_len >= 200 and seq_len <= 3000:
                        seq_tuple.append((header, sequence))
                header = line[1:]
                sequence = ''
                count += 1
                
            else:
                sequence += line
                
        # Check the last sequence
        seq_len = len(sequence)
        if seq_len >= 300 and seq_len <= 1000:
            seq_tuple.append((header, sequence))
        return seq_tuple, count

In [6]:
coding = "coding.fa"
non_coding = "noncoding.fa"

In [7]:
coding_list, coding_count = load_file(coding)  # function call on coding dataset

In [8]:
 noncoding_list, noncoding_count = load_file(non_coding)   # function call on noncoding dataset

In [9]:
len(noncoding_list)

58022

In [10]:
len(coding_list)

111275

In [11]:
coding_count

111276

In [12]:
noncoding_count

58023

# Generate dataset

## 1. Fickett Score

In [13]:
# NAR 10(17) 5303-531
position_parameter = [1.9,1.8,1.7,1.6,1.5,1.4,1.3,1.2,1.1,0.0]
position_prob_coding = {
'A':[0.94,0.68,0.84,0.93,0.58,0.68,0.45,0.34,0.20,0.22],
'C':[0.80,0.70,0.70,0.81,0.66,0.48,0.51,0.33,0.30,0.23],
'G':[0.90,0.88,0.74,0.64,0.53,0.48,0.27,0.16,0.08,0.08],
'T':[0.97,0.97,0.91,0.68,0.69,0.44,0.54,0.20,0.09,0.09]
}
position_weight = {'A':0.26,'C':0.18,'G':0.31,'T':0.33}

content_parameter = [0.33,0.31,0.29,0.27,0.25,0.23,0.21,0.17,0]
content_prob_coding = {
'A':[0.28,0.49,0.44,0.55,0.62,0.49,0.67,0.65,0.81,0.21],
'C':[0.82,0.64,0.51,0.64,0.59,0.59,0.43,0.44,0.39,0.31],
'G':[0.40,0.54,0.47,0.64,0.64,0.73,0.41,0.41,0.33,0.29],
'T':[0.28,0.24,0.39,0.40,0.55,0.75,0.56,0.69,0.51,0.58]
}
content_weight = {'A':0.11,'C':0.12,'G':0.15,'T':0.14}

def position_map(pos_value, base):
    if pos_value < 0:
        return None
    for i,v in enumerate(position_parameter):
        if pos_value >= v:
            posbyweight = position_prob_coding[base][i]*position_weight[base]
            return posbyweight

def content_map(con_value, base):
    if con_value < 0:
        return None
    for i,v in enumerate(content_parameter):
        if con_value >= v:
            conbyweight = content_prob_coding[base][i]*content_weight[base]
            return conbyweight

def fickett(sequence):
    A123pos = [0, 0, 0]  # A1, A2, A3
    T123pos = [0, 0, 0]  # T1, T2, T3
    G123pos = [0, 0, 0]  # G1, G2, G3
    C123pos = [0, 0, 0]  # C1, C2, C3
    fickett_score = 0
    for i in range(3):
        for j in range(i, len(sequence), 3):
            if sequence[j] == 'A':
                A123pos[i] += 1
            elif sequence[j] == 'T':
                T123pos[i] += 1
            elif sequence[j] == 'G':
                G123pos[i] += 1
            elif sequence[j] == 'C':
                C123pos[i] += 1
    # A, T, G, C positions
    Apos = max(A123pos[0], A123pos[1], A123pos[2]) / (min(A123pos[0], A123pos[1], A123pos[2]) + 1)
    Tpos = max(T123pos[0], T123pos[1], T123pos[2]) / (min(T123pos[0], T123pos[1], T123pos[2]) + 1)
    Gpos = max(G123pos[0], G123pos[1], G123pos[2]) / (min(G123pos[0], G123pos[1], G123pos[2]) + 1)
    Cpos = max(C123pos[0], C123pos[1], C123pos[2]) / (min(C123pos[0], C123pos[1], C123pos[2]) + 1)
    # A, T, G, C content
    Acon = sequence.count('A') / len(sequence)
    Tcon = sequence.count('T') / len(sequence)
    Gcon = sequence.count('G') / len(sequence)
    Ccon = sequence.count('C') / len(sequence)

    # fickett score
    fickett_score += position_map(Apos, 'A')
    fickett_score += position_map(Tpos, 'T')
    fickett_score += position_map(Gpos, 'G')
    fickett_score += position_map(Cpos, 'C')

    fickett_score += content_map(Acon, 'A')
    fickett_score += content_map(Tcon, 'T')
    fickett_score += content_map(Gcon, 'G')
    fickett_score += content_map(Ccon, 'C')
    return fickett_score


## 2.1 ORF Length - T0

In [14]:
def translation(sequence):
    '''
    :param sequence: input primary sequence of a transcript
    :return: protein sequence
    '''
    trans_dic = {'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'CTT': 'L',
                 'CTC': 'L', 'CTA': 'L', 'CTG': 'L', 'ATT': 'I', 'ATC': 'I',
                 'ATA': 'I', 'ATG': 'M', 'GTT': 'V', 'GTC': 'V', 'GTA': 'V',
                 'GTG': 'V', 'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
                 'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P', 'ACT': 'T',
                 'ACC': 'T', 'ACA': 'T', 'ACG': 'T', 'GCT': 'A', 'GCC': 'A',
                 'GCA': 'A', 'GCG': 'A', 'TAT': 'Y', 'TAC': 'Y', 'TAA': '*',
                 'TAG': '*', 'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
                 'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'GAT': 'D',
                 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'TGT': 'C', 'TGC': 'C',
                 'TGA': '*', 'TGG': 'W', 'CGT': 'R', 'CGC': 'R', 'CGA': 'R',
                 'CGG': 'R', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
                 'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}
    protein = ''
    for i in range(0, len(sequence)-2, 3):
        protein += trans_dic[sequence[i:i+3]]
    return protein

In [15]:
def ORFfinder_T0(sequence):
    # 3 frame translation
    trans1 = translation(sequence)
    trans2 = translation(sequence[1:])
    trans3 = translation(sequence[2:])

    orf1 = re.finditer(r'M.*?\*', trans1)
    orf2 = re.finditer(r'M.*?\*', trans2)
    orf3 = re.finditer(r'M.*?\*', trans3)
    orf1_seqs = [(m.start(), m.end(), m.group(), 'frame1') for m in orf1]
    orf2_seqs = [(m.start(), m.end(), m.group(), 'frame2') for m in orf2]
    orf3_seqs = [(m.start(), m.end(), m.group(), 'frame3') for m in orf3]

    orfs = orf1_seqs + orf2_seqs + orf3_seqs
    if len(orfs) == 0:
#         return ('NaN', 'NaN'), 'NaN', 0
        return 0
    else:
        orf_sorted = sorted(orfs, key=lambda t: len(t[2]), reverse=True)
        longest_orf_protein_seq = orf_sorted[0][2]
        longest_orf_frame = orf_sorted[0][3]
        longest_orf_pro_position = (int(orf_sorted[0][0]), int(orf_sorted[0][1]))
        if longest_orf_frame == 'frame1':
            longest_orf_rna_seq_position = (longest_orf_pro_position[0] * 3, longest_orf_pro_position[1] * 3)
            longest_orf_rna_seq = sequence[longest_orf_rna_seq_position[0]:longest_orf_rna_seq_position[1]]
            longest_orf_rna_length = longest_orf_rna_seq_position[1] - longest_orf_rna_seq_position[0]
            return longest_orf_rna_length
#             return longest_orf_rna_seq_position, longest_orf_rna_seq, longest_orf_rna_length
        elif longest_orf_frame == 'frame2':
            longest_orf_rna_seq_position = (longest_orf_pro_position[0] * 3, longest_orf_pro_position[1] * 3)
            longest_orf_rna_seq = sequence[1:][longest_orf_rna_seq_position[0]:longest_orf_rna_seq_position[1]]
            longest_orf_rna_seq_original_position = (
            longest_orf_pro_position[0] * 3 + 1, longest_orf_pro_position[1] * 3 + 1)
            longest_orf_rna_length = longest_orf_rna_seq_original_position[1] - longest_orf_rna_seq_original_position[0]
            return longest_orf_rna_length
#             return longest_orf_rna_seq_original_position, longest_orf_rna_seq, longest_orf_rna_length
        elif longest_orf_frame == 'frame3':
            longest_orf_rna_seq_position = (longest_orf_pro_position[0] * 3, longest_orf_pro_position[1] * 3)
            longest_orf_rna_seq = sequence[2:][longest_orf_rna_seq_position[0]:longest_orf_rna_seq_position[1]]
            longest_orf_rna_seq_original_position = (
            longest_orf_pro_position[0] * 3 + 2, longest_orf_pro_position[1] * 3 + 2)
            longest_orf_rna_length = longest_orf_rna_seq_original_position[1] - longest_orf_rna_seq_original_position[0]
            return longest_orf_rna_length
#             return longest_orf_rna_seq_original_position, longest_orf_rna_seq, longest_orf_rna_length



## 2.2 T1 - type 1 ORF length

In [16]:
def ORFfinder_T1(sequence):
    end = len(sequence)
    start = end
    current_orf_len = 0
    current_orf_seq = ''
    current_position = (0,0)
    for frame in range(3):
        for i in range(frame,len(sequence),3):
            codon = sequence[i:i+3]
            if codon == 'ATG':
                start = i
                break
            else:
                continue
        current_len = end - start
        if current_len >= current_orf_len:
            current_orf_len = current_len
            current_orf_seq = sequence[start:end]
            current_position = (start, end)
    if int(current_orf_len) == 0:
        return ('NaN','NaN'),'NaN', 0
    else:
        return current_orf_len


## 2.3 T2 - Type 2 ORF Length

In [17]:
def ORFfinder_T2(sequence):
    start = 0
    end = 0
    current_orf_len = 0
    current_orf_seq = ''
    current_position = (0,0)
    for frame in range(3):
        for i in range(frame,len(sequence),3):
            codon = sequence[i:i+3]
            if codon in ['TAG','TGA','TAA']:
                end = i+3
                current_len = end - start
                if current_len >= current_orf_len:
                    current_orf_len = current_len
                    current_orf_seq = sequence[start:end]
                    current_position = (start, end)
                start = end
            else:
                continue
    if int(current_orf_len) == 0:
        return ('NaN','NaN'),'NaN', 0
    else:
        return current_orf_len

## 2.4 T3 - Type 3 ORF Length

In [18]:
def ORFfinder_T3(sequence):
    end = len(sequence)
    start = end
    current_orf_len = 0
    current_orf_seq = ''
    current_position = (0, 0)
    for frame in range(3):
        for i in range(frame, len(sequence), 3):
            codon = sequence[i:i + 3]
            if codon == 'ATG':
                start = i
                break
            else:
                continue
        current_len = end - start
        if current_len >= current_orf_len:
            current_orf_len = current_len
            current_orf_seq = sequence[start:end]
            current_position = (start, end)

    start = 0
    end = 0
    for frame in range(3):
        for i in range(frame, len(sequence), 3):
            codon = sequence[i:i + 3]
            if codon in ['TAG', 'TGA', 'TAA']:
                end = i + 3
                current_len = end - start
                if current_len >= current_orf_len:
                    current_orf_len = current_len
                    current_orf_seq = sequence[start:end]
                    current_position = (start, end)
                start = end
            else:
                continue

    if int(current_orf_len) == 0:
        return ('NaN', 'NaN'), 'NaN', 0
    else:
        return current_orf_len


## 3. Relative Codon Bias

In [19]:
    
def RCB_score(seq):
    condon_dict = {}         # a dictionary contain the counts of each condon
    condon_list = []         # a list contain the condons in a sequence
    Frequency_1 = {'A': 0,
                'T': 0,
                'G' : 0,
                'C' : 0}
    Frequency_2 = {'A': 0,
                'T': 0,
                'G' : 0,
                'C' : 0}
    Frequency_3 = {'A': 0,
                'T': 0,
                'G' : 0,
                'C' : 0}

    # First position
    for i in range(0,len(seq),3):
        if seq[i] == 'A':
            Frequency_1['A'] += 1
        elif seq[i] == 'T':
            Frequency_1['T'] += 1
        elif seq[i] == 'G':
            Frequency_1['G'] += 1
        elif seq[i] == 'C':
            Frequency_1['C'] += 1

        if len(seq[i:i+3]) != 3:
            continue
        elif seq[i:i+3] not in condon_dict:
            condon_dict[seq[i:i+3]] = 1
        else:
            condon_dict[seq[i:i+3]] += 1

        if len(seq[i:i+3]) == 3:
            condon_list.append(seq[i:i+3])

    # Second position
    for i in range(1,len(seq),3):
        if seq[i] == 'A':
            Frequency_2['A'] += 1
        elif seq[i] == 'T':
            Frequency_2['T'] += 1
        elif seq[i] == 'G':
            Frequency_2['G'] += 1
        elif seq[i] == 'C':
            Frequency_2['C'] += 1

    # Third position
    for i in range(2,len(seq),3):
        if seq[i] == 'A':
            Frequency_3['A'] += 1
        elif seq[i] == 'T':
            Frequency_3['T'] += 1
        elif seq[i] == 'G':
            Frequency_3['G'] += 1
        elif seq[i] == 'C':
            Frequency_3['C'] += 1

    total_condons = len(condon_list)
    condon_sum = 0
    for k in condon_list:
        d = np.log(np.absolute((condon_dict[k]/total_condons) - (Frequency_1[k[0]]/total_condons) *
             (Frequency_2[k[1]]/total_condons) * (Frequency_3[k[2]]/total_condons)) / ((Frequency_1[k[0]]/total_condons) *
             (Frequency_2[k[1]]/total_condons) * (Frequency_3[k[2]]/total_condons)) + 1)
        condon_sum = condon_sum + d

    RCB = np.exp(condon_sum/len(condon_list))-1
    return RCB



## 4.  Isoelectric Point

In [20]:
def cal_protein_features_pi(seq):
    '''
    :param seq: input primary sequence of a transcript
    :return: pi: (protein isoelectric),
             mw: (molecular weight)
             # gravy: (grand average of hydropathy)
             aromaticity: (relative frequency of Phe+Trp+Tyr)
             instability: (instability index --
                          Any value above 40 means the protein is unstable (=has a short half life)).
    '''
    protein_seq = translation(seq).strip('*')
    protein_object = ProtParam.ProteinAnalysis(protein_seq)
    pi = protein_object.isoelectric_point()
    return pi

## 5. Aromaticity

In [21]:
def cal_protein_features_aromaticity(seq):
    '''
    :param seq: input primary sequence of a transcript
    :return: pi: (protein isoelectric),
             mw: (molecular weight)
             # gravy: (grand average of hydropathy)
             aromaticity: (relative frequency of Phe+Trp+Tyr)
             instability: (instability index --
                          Any value above 40 means the protein is unstable (=has a short half life)).
    '''

    protein_seq = translation(seq).strip('*')
    protein_object = ProtParam.ProteinAnalysis(protein_seq)
    aromaticity = protein_object.aromaticity()
    return aromaticity

## 6. Transcript Length

In [22]:
def transcript_length(seq):
    return len(seq)

## 7. ORF Length

**ORF length is the actual length of the coding sequence in nucleotides.**

## 8. ORF Coverage

**ORF coverage refers to the percentage of the entire sequence that is covered by the ORF.
<br> For example, if an ORF is 300 nucleotides long and the entire sequence is 1000 nucleotides long, then the ORF length is 300 nucleotides and the ORF coverage is 30% (300/1000).**

## 9. CpG Islands

In [23]:
def calculate_cpg_islands(seq):
    cpg_islands = []
    cpg_count = 0
    in_island = False
    
    for i in range(len(seq)-1):
        if seq[i:i+2].upper() == 'CG':
            cpg_count += 1
            if not in_island:
                in_island = True
        else:
            if cpg_count > 0:
                cpg_islands.append(cpg_count)
                cpg_count = 0
                in_island = False
    
    if cpg_count > 0:
        cpg_islands.append(cpg_count)
    
    return cpg_islands

## 10. GC content

In [24]:
def gcContent(sequence):
    '''
    :param sequence: primary sequence of a transcript
    :return: the GC content of the sequence
    '''
    Gcontent = 0
    Ccontent = 0
    for base in sequence:
        if base == 'G':
            Gcontent += 1
        if base == 'C':
            Ccontent += 1
    GCcontent = (Gcontent+Ccontent)/len(sequence)
    return GCcontent

## Generating feature arrays

In [25]:
import pandas as pd

In [43]:


def generate_feature(list_name):
    
    
    for header, sequence in list_name:        
        #calculate fickett score
        fickett_score = fickett(sequence)
        
        #calculate GC content
        gc = gcContent(sequence)
        
        # Calculate the number of CpG islands for this sequence
        cpg_islands = calculate_cpg_islands(sequence)
        num_islands = len(cpg_islands)
    
        #calculate transcript length
        length = transcript_length(sequence)
        
        #calculate ORF length
        orf_length = ORFfinder_T0(sequence)
        
        #calculate T1
        t1 = ORFfinder_T1(sequence)
        
        #calculate T2
        t2 = ORFfinder_T2(sequence)
        
        #calculate T3
        t3 = ORFfinder_T3(sequence)
        
        #calculate ORF ratio
        orf_ratio = orf_length/length
        
        #calculate Relative Codon Bias , isoelectric point, aromaticity
        if orf_length != 0:
            rcb_t0 = RCB_score(sequence)
            pi = cal_protein_features_pi(sequence)
            arom = cal_protein_features_aromaticity(sequence)
        else:
            rcb_t0 = 0
            pi = 0
            arom = 0
        
        fickett_array.append(fickett_score)
        gc_array.append(gc)
        cpg_array.append(num_islands)
        transcript_array.append(length)
        orf_array.append(orf_length)
        orf_ratio_array.append(orf_ratio)
        rcb_array.append(rcb_t0)
        ip_array.append(pi)
        t1_array.append(t1)
        t2_array.append(t2)
        t3_array.append(t3)
        aromaticity_array.append(arom)   
        
    

#  Create coding dataset

In [44]:
fickett_array = []
gc_array = []
orf_array = []
rcb_array=[]
ip_array = []
aromaticity_array = []
transcript_array = []
orf_ratio_array = []
cpg_array = []
t1_array = []
t2_array = []
t3_array = []
generate_feature(coding_list) # generate feature set for coding dataset

In [45]:
names_dict = {'Fickett Score':fickett_array,'GC content':gc_array, 'CpG Islands':cpg_array, 'Transcript Length':transcript_array, 'ORF Length':orf_array, 'T1': t1_array, 'T2':t2_array, 'T3':t3_array,'ORF Ratio':orf_array, 'Relative Codon Bias':rcb_array, 'Isoelectric Potential':ip_array, 'Aromaticity':aromaticity_array}
features_df = pd.DataFrame(names_dict)

In [46]:
features_df.head()

,Fickett Score,GC content,CpG Islands,Transcript Length,ORF Length,T1,T2,T3,ORF Ratio,Relative Codon Bias,Isoelectric Potential,Aromaticity
0,0.3871,0.359435,18,2618,981,2598,993,2598,981,0.284229,9.588915,0.112385
1,0.9340,0.460064,8,939,939,939,939,939,939,0.460470,9.042610,0.137821
2,0.9340,0.460064,8,939,939,939,939,939,939,0.460470,9.042610,0.137821
3,1.0718,0.699278,302,3465,2535,3173,2979,3173,2535,0.270346,11.870000,0.051948
4,1.0718,0.699250,302,3468,2538,3176,2982,3176,2538,0.271500,11.870000,0.051903


In [47]:
t1_array


[2598,
 939,
 939,
 3173,
 3176,
 327,
 2467,
 1722,
 1860,
 2049,
 1671,
 1986,
 2177,
 1696,
 2741,
 2457,
 878,
 3126,
 2164,
 2425,
 679,
 3488,
 3003,
 2495,
 1032,
 761,
 659,
 681,
 558,
 560,
 7273,
 7344,
 7217,
 7148,
 1600,
 2318,
 1767,
 1939,
 2034,
 3149,
 2218,
 1000,
 771,
 824,
 723,
 504,
 525,
 442,
 2206,
 2206,
 435,
 1959,
 243,
 851,
 1062,
 727,
 705,
 1126,
 1006,
 1049,
 1158,
 1041,
 1127,
 785,
 852,
 920,
 933,
 977,
 1909,
 2025,
 2041,
 721,
 2763,
 1004,
 2267,
 2071,
 2198,
 862,
 1057,
 819,
 853,
 844,
 671,
 305,
 736,
 646,
 578,
 304,
 299,
 738,
 2983,
 2237,
 3470,
 2645,
 583,
 2529,
 841,
 3678,
 2280,
 576,
 1210,
 436,
 2550,
 2184,
 1783,
 2062,
 1767,
 1839,
 2529,
 1753,
 1598,
 1777,
 1946,
 1484,
 807,
 1034,
 609,
 527,
 832,
 687,
 749,
 1046,
 422,
 497,
 502,
 507,
 574,
 1850,
 484,
 3466,
 2879,
 3173,
 614,
 775,
 2231,
 2242,
 1962,
 2635,
 671,
 669,
 665,
 664,
 3083,
 2024,
 2810,
 1576,
 1175,
 642,
 458,
 682,
 541,
 1034,
 

In [48]:
t3_array

[2598,
 939,
 939,
 3173,
 3176,
 327,
 2467,
 1722,
 1860,
 2049,
 1671,
 1986,
 2177,
 1696,
 2741,
 2457,
 878,
 3126,
 2164,
 2425,
 679,
 3488,
 3003,
 2495,
 1032,
 762,
 659,
 681,
 558,
 560,
 7273,
 7344,
 7217,
 7148,
 1600,
 2318,
 1767,
 1939,
 2034,
 3149,
 2218,
 1000,
 771,
 824,
 723,
 504,
 525,
 442,
 2206,
 2206,
 435,
 1959,
 243,
 851,
 1062,
 727,
 705,
 1126,
 1006,
 1049,
 1158,
 1041,
 1127,
 785,
 852,
 920,
 933,
 977,
 1909,
 2025,
 2041,
 721,
 2763,
 1004,
 2267,
 2071,
 2198,
 862,
 1057,
 1008,
 996,
 844,
 671,
 305,
 736,
 646,
 578,
 325,
 299,
 738,
 2983,
 2237,
 3470,
 2645,
 583,
 2529,
 841,
 3678,
 2280,
 576,
 1210,
 495,
 2550,
 2184,
 1783,
 2062,
 1767,
 1839,
 2529,
 1753,
 1598,
 1777,
 1946,
 1484,
 807,
 1034,
 609,
 527,
 832,
 687,
 749,
 1046,
 422,
 497,
 502,
 507,
 574,
 1850,
 484,
 3466,
 2879,
 3173,
 654,
 775,
 2231,
 2242,
 1962,
 2635,
 671,
 729,
 665,
 675,
 3083,
 2024,
 2810,
 1576,
 1175,
 642,
 458,
 682,
 541,
 1034,


### Adding a label for coding sequences

In [49]:
features_df["coding/noncoding"] = 1    #1 for coding sequences
features_df.head()

,Fickett Score,GC content,CpG Islands,Transcript Length,ORF Length,T1,T2,T3,ORF Ratio,Relative Codon Bias,Isoelectric Potential,Aromaticity,coding/noncoding
0,0.3871,0.359435,18,2618,981,2598,993,2598,981,0.284229,9.588915,0.112385,1
1,0.9340,0.460064,8,939,939,939,939,939,939,0.460470,9.042610,0.137821,1
2,0.9340,0.460064,8,939,939,939,939,939,939,0.460470,9.042610,0.137821,1
3,1.0718,0.699278,302,3465,2535,3173,2979,3173,2535,0.270346,11.870000,0.051948,1
4,1.0718,0.699250,302,3468,2538,3176,2982,3176,2538,0.271500,11.870000,0.051903,1


In [50]:
features_df.shape

(111275, 13)

In [52]:
features_df.to_csv('coding13.csv', index=False)

# Create non-coding dataset

In [27]:
fickett_array = []
gc_array = []
orf_array = []
rcb_array=[]
ip_array = []
aromaticity_array = []
transcript_array = []
orf_ratio_array = []
cpg_array = []
t1_array = []
t2_array = []
t3_array = []
generate_feature(noncoding_list) # generate feature set for coding dataset

In [28]:
names_dict = {'Fickett Score':fickett_array,'GC content':gc_array, 'CpG Islands':cpg_array, 'Transcript Length':transcript_array, 'ORF Length':orf_array,'T1': t1_array, 'T2':t2_array, 'T3':t3_array, 'ORF Ratio':orf_array, 'Relative Codon Bias':rcb_array, 'Isoelectric Potential':ip_array, 'Aromaticity':aromaticity_array}
features_df = pd.DataFrame(names_dict)

In [29]:
features_df.head()

,Fickett Score,GC content,CpG Islands,Transcript Length,ORF Length,ORF Ratio,Relative Codon Bias,Isoelectric Potential,Aromaticity
0,0.6045,0.553410,23,1657,231,231,0.458169,8.878344,0.068841
1,0.5691,0.585674,32,712,228,228,0.497487,9.470164,0.067511
2,0.4430,0.467290,11,535,111,111,0.518787,8.080870,0.101124
3,0.6268,0.533277,20,1187,258,258,0.387685,9.154591,0.058228
4,0.6034,0.535593,9,590,225,225,0.580721,9.527154,0.102041


## Adding a label for non-coding sequences

In [30]:
features_df["coding/noncoding"] = 0    #0 for coding sequences
features_df.head()

,Fickett Score,GC content,CpG Islands,Transcript Length,ORF Length,ORF Ratio,Relative Codon Bias,Isoelectric Potential,Aromaticity,coding/noncoding
0,0.6045,0.553410,23,1657,231,231,0.458169,8.878344,0.068841,0
1,0.5691,0.585674,32,712,228,228,0.497487,9.470164,0.067511,0
2,0.4430,0.467290,11,535,111,111,0.518787,8.080870,0.101124,0
3,0.6268,0.533277,20,1187,258,258,0.387685,9.154591,0.058228,0
4,0.6034,0.535593,9,590,225,225,0.580721,9.527154,0.102041,0


In [31]:
features_df.shape

(58022, 10)

In [32]:
features_df.to_csv('noncoding_9_features.csv', index=False)